# Clean Go at Par Formation

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
from datetime import datetime
from datetime import date

## Read the file in, from the "inputs" folder to Pandas

In [2]:
#DEAR -> REPORTS -> SALES BY PRODUCT DETAILS -> [select time frame] -> EXPORT -> EXCEL WITHOUT TOTALS

df = pd.read_excel('inputs/31may22last365.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)#parse_dates didn't work, see below

## Start Cleaning and Featuring

In [3]:
#initial cleaning setup
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible
list_of_retail_stores = ['Customer Square ' + str(x+1) for x in range(len(retail_names))] #we have 3 stores active, we won't count the Warehouse
store_map = dict(zip(list_of_retail_stores,retail_names))

#ws = df[df.Customer.isin(list_of_retail_stores) == False] #WHOLESALE
df = df[df.Customer.isin(list_of_retail_stores) & df.Product.notnull()].reset_index(drop=True) #remove wholesale and weird null anomoly entries

### Check for duplicate products / DEAR errors (Backend inventory management system)

In [4]:
#df['SKU'] = df.SKU.apply(str)
temp = df.groupby(["Product","SKU"]).count().reset_index()
temp[temp['Product'].duplicated()]

,Product,SKU,Category,Invoice Date,Customer,Quantity,Sale,COGS,Profit


### Features Adding and Dropping

In [5]:
#start cleaning some of the unused or needed columns / rows, converting "Invoice Date" to datetime objects
df = df.drop(['SKU', 'COGS','Profit'], axis=1, inplace=False)#not trusted data columns or useful, we'll ignore them for now
df = df.rename(columns={'Invoice Date':'Date'})#renaming for legibility 

In [6]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')#get these dates converted from string to something more powerful
df['Month'] = df.Date.apply(lambda x: x.month)#.month_name() also an option but i dont trust yet
df['Weekday'] = df.Date.apply(lambda x: x.dayofweek)#Monday is 0, Sunday is 6, day_name() also an option but i dont trust yet
df['Week'] = df.Date.apply(lambda x: x.isocalendar()[1])#x.isocalendar().week
df["Unit"] = df.Sale / df.Quantity
df.head(2)

,Product,Category,Date,Customer,Quantity,Sale,Month,Weekday,Week,Unit
0,Matthiasson Cabernet Sauvignon 750ml,Red New World,2021-07-03,Customer Square 3,1,67.99,7,5,26,67.99
1,Matthiasson Cabernet Sauvignon 750ml,Red New World,2021-08-09,Customer Square 2,1,67.99,8,0,32,67.99


In [7]:
itnl_whiskey = ['Whiskey French', 'Whiskey Indian', 'Whiskey Mexican', 'Whiskey Taiwaneese', 'Whisky German']
df['Category'] = df.apply(lambda row: "Itnl Whiskey" if row.Category in itnl_whiskey else row.Category,axis=1)
#df[df.Category == 'Itnl Whiskey']

In [8]:
#df[(df.Product == 'Del Maguey Vida de Muertos750ml') & (df.Customer == 'Customer Square 2')]

## Make the Par Suggestion DataFrame

In [9]:
df[df.Week == 53].head(5)

,Product,Category,Date,Customer,Quantity,Sale,Month,Weekday,Week,Unit
60562,El Guapo Tex Mex Bitters 100ml,Bitters,2020-12-31,Customer Square 2,0,0.0,12,3,53,NaN


In [10]:
test = pd.pivot_table(df[df.Customer == 'Customer Square 2'], index = 'Product', columns = 'Week', values = 'Quantity', aggfunc = 'sum')
#test.head(2)

In [11]:
test = test.reset_index().drop(columns=[53])
test = test.fillna(0)
test.head(2)

Week,Product,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,123 Organic Anejo Tequila 750ml,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,123 Organic Blanco Tequila 750ml,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [12]:
#test = df[df.Product != 'eGift Card'].groupby(['Product','Week','Customer']).Quantity.sum().reset_index()
test[test.Product == 'Del Maguey Vida de Muertos750ml']

Week,Product,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
1041,Del Maguey Vida de Muertos750ml,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,2.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


Not sure if I need to get rid of eGiftCards...\
Anyways there's gotta be a better way to do this next cell of code, but basically for each row we want to run some statistical info gathering for the series of sales per week per item. Right now, it's a basic for-loop, which doesn't seem very P Y T H O N I C. I'll continue to look for other, more flexible options or packages or whatever.

In [13]:
gather_frame = []
for i in range(len(test)):
    a = test.iloc[i,1:].to_numpy().flatten()
    r = [a.sum(),a.mean(),a.std(),a.min(),a.max(),np.percentile(a,50),np.percentile(a,75),np.percentile(a,85)]
    gather_frame.append(list(r))

In [14]:
stat_frame = pd.DataFrame(gather_frame, columns = ['Total','Mean','Std','Min','Max','50','75','85'])
here_we_go = pd.concat([test,stat_frame],axis=1)
here_we_go = here_we_go.drop(columns=range(1,53))
here_we_go

,Product,Total,Mean,Std,Min,Max,50,75,85
0,123 Organic Anejo Tequila 750ml,5.0,0.096154,0.354076,0.0,2.0,0.0,0.0,0.00
1,123 Organic Blanco Tequila 750ml,11.0,0.211538,0.453047,0.0,2.0,0.0,0.0,1.00
2,123 Organic Reposado Tequila 750ml,8.0,0.153846,0.360801,0.0,1.0,0.0,0.0,0.35
3,17/24 Metodo Classico 750ml,24.0,0.461538,0.795698,0.0,3.0,0.0,1.0,1.00
4,1792 Bottled In Bond 750ml,14.0,0.269231,0.682625,0.0,3.0,0.0,0.0,1.00
...,...,...,...,...,...,...,...,...,...
3743,Zing Zang Bloody Mary Mix 1.75L,4.0,0.076923,0.549341,0.0,4.0,0.0,0.0,0.00
3744,Zing Zang Bloody Mary Mix PET 32oz,31.0,0.596154,0.945833,0.0,4.0,0.0,1.0,1.00
3745,Zubrowka Bison Grass Vodka 750ml,37.0,0.711538,0.926875,0.0,3.0,0.0,1.0,2.00
3746,Zucca Rabarbaro 750ml,10.0,0.192308,0.394113,0.0,1.0,0.0,0.0,1.00


## Read in current pars

In [15]:
#DEAR -> PRODUCTS -> EXPORT -> STOCK REORDER LOCATIONS

read_pars = pd.read_csv('inputs/StockReorderlocations_2022-05-31.csv',header=0,dtype=str)#parse_dates
read_pars.rename(columns={'ProductName':'Product','Location':'Customer'},inplace=True)
read_pars.drop(['StockLocator','PickZones'],inplace=True,axis=1)
read_pars.MinimumBeforeReorder = read_pars.MinimumBeforeReorder.astype(float)
read_pars.MinimumBeforeReorder = read_pars.MinimumBeforeReorder.astype(int)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(float)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(int)
#read_pars.info()

### Location -> Customer

In [16]:
fixer = dict(zip(retail_names,list_of_retail_stores))
read_pars['Customer'] = read_pars.apply(lambda x: fixer[x.Customer] if x.Customer != 'Warehouse' else 'Warehouse',axis=1 )
read_pars = read_pars[read_pars.Customer == 'Customer Square 2']
#read_pars

### Merge the two frames, create new columns / features

In [17]:
#print(par_suggest.head())
#print(read_pars.head())
mess = here_we_go.merge(right=read_pars,on=['Product'])
mess['Unit'] = mess.apply(lambda x: df.loc[df.Product == x.Product].iloc[0].Unit,axis=1)
mess['RoundedMean'] = mess['Mean'].round()
mess['DiffMean'] = (mess['MinimumBeforeReorder'] - mess['RoundedMean']).round() 
mess['Diff75'] = (mess['MinimumBeforeReorder'] - mess['75']).round()
mess['Diff85'] = (mess['MinimumBeforeReorder'] - mess['85']).round()
mess['MeanSavings'] = mess['DiffMean'] * mess['Unit']
mess['75Savings'] = mess['Diff75'] * mess['Unit']
mess['85Savings'] = mess['Diff85'] * mess['Unit']
mess.to_csv('outputs/paranalysis.csv',index=False)
#print(mess.info())
mess.head()

,Product,Total,Mean,Std,Min,Max,50,75,85,Action,...,MinimumBeforeReorder,ReorderQuantity,Unit,RoundedMean,DiffMean,Diff75,Diff85,MeanSavings,75Savings,85Savings
0,123 Organic Blanco Tequila 750ml,11.0,0.211538,0.453047,0.0,2.0,0.0,0.0,1.00,Create/Update,...,1,3,47.99,0.0,1.0,1.0,0.0,47.99,47.99,0.00
1,123 Organic Reposado Tequila 750ml,8.0,0.153846,0.360801,0.0,1.0,0.0,0.0,0.35,Create/Update,...,1,3,56.99,0.0,1.0,1.0,1.0,56.99,56.99,56.99
2,17/24 Metodo Classico 750ml,24.0,0.461538,0.795698,0.0,3.0,0.0,1.0,1.00,Create/Update,...,2,6,42.99,0.0,2.0,1.0,1.0,85.98,42.99,42.99
3,1792 Bottled In Bond 750ml,14.0,0.269231,0.682625,0.0,3.0,0.0,0.0,1.00,Create/Update,...,2,6,38.99,0.0,2.0,2.0,1.0,77.98,77.98,38.99
4,1792 Full Proof 750ml,6.0,0.115385,0.423077,0.0,2.0,0.0,0.0,0.00,Create/Update,...,1,6,45.99,0.0,1.0,1.0,1.0,45.99,45.99,45.99


## Export results as an uploadable .csv

In [18]:
#let's generate something we can upload to DEAR
par_gen = mess[['Action','ProductSKU','Product','Customer','75','85','ReorderQuantity','75Savings','85Savings']].copy()
par_gen = par_gen.rename(columns={'Product':'ProductName','Customer':'Location','75':'MinimumBeforeReorder75','85':'MinimumBeforeReorder85'})

In [19]:
par_gen = par_gen[par_gen.Location == 'Customer Square 2']
par_gen.MinimumBeforeReorder75 = par_gen.MinimumBeforeReorder75.round().astype(int)
par_gen.MinimumBeforeReorder85 = par_gen.MinimumBeforeReorder85.round().astype(int)
par_gen['Stock Locator'] = par_gen['PickZones'] = ''
par_gen.to_csv('outputs/PVParGenerator2.csv',index=False)
#par_gen.info()